In [1]:
from manager.TaskManager import TaskManager

# Time-based reasoning
manager = TaskManager(
    agents=["Deepseek_R1_7B_Qwen"],
    rounds=[3,4,5],
    quadrants=[4],
    reasoning_mode="time",
    min_thinking_time=5.0,
    max_thinking_time=15.0,
    n_simulations=3,
    n_runs=1,
    verbose=True,
    device="cuda:0"
)

# Token-based reasoning
# manager = TaskManager(
#     agents=["Deepseek_R1_7B_Qwen"],
#     rounds=[4],
#     quadrants=[4],
#     reasoning_mode="tokens",
#     min_thinking_tokens=100,
#     max_thinking_tokens=300,
#     n_simulations=1,
#     n_runs=1,
#     verbose=True,
#     device="cuda:0"
# )

# Run benchmarks
results = manager.multiple_benchmarks()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-26 01:14:59 __init__.py:190] Automatically detected platform cuda.
Running benchmark for agent: Deepseek_R1_7B_Qwen
Using unsloth with GPU
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.621 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Running benchmark for Deepseek_R1_7B_Qwen, 3 rounds, 4 quadrants, run 1/1

Simulation 1/3

=== Task Description ===
You will play a game with 3 rounds.
In each round you'll see active cues (chooseable):
One quadrant has 90% one color/10% other
Other quadrants have 50/50 color distribution
At least one cue active per round
Active cues disappear after random duration

After 3 rounds, identify the biased quadrant.
Correct: +100 points, Wrong: -100 points.

=== Beginning Rounds ===

--- Round 1 ---

Accumulated prompt shown to LLM:
--------------------
You will play a game with 3 rounds.
In each round you'll see active cues (chooseable):
One quadrant has 90% one color/10% other
Other quadrants have 50/50 color distribution
At least one cue active per round
Active cues disappear after random duration

After 3 rounds, identify the biased quadrant.
Correct: +100 points, Wrong: -100 points.

Round 1: Available cues A, C, D. Based on previous observations, choose one cue by responding with just